In [16]:
codes = {
    'XCSH': '10',
    'DET' : '00',
    'QPL' : '2',
    'MBPL' : '8',
    'drift': '1',
    'TGT': '20',
    'END': '20'
}

#For elements like magnets (yoke + field) the list is [lengthFull, lengthCore], otherwise just length
lengths = {
    'XCSH': 1.1,
    'DET' : 6, 
    'QPL' : [2.46, 2.00],
    'MBPL' : [2.67, 2.00],
    'TGT': 0.20,
    'END': 0.00
}


In [2]:
def GetBEATCHString(element):
    
    #Hard coded lengths of gaps for right formatting of strings
    angleSpaces = 13
    rotationSpaces = 21

    string = element.Name.ljust(18, ' ') + element.Code.rjust(2, ' ') + ('%4.3f' %element.Length).rjust(7, ' ')

    #Difference between positive and negative so that the first digit of x and -x alight instead of the - aligning
    if element.Angle > 0:
        string = string  + (str(element.Angle)).ljust(9, ' ').rjust(angleSpaces, ' ')

    elif element.Angle < 0:
        string = string  + (str(element.Angle)).ljust(10, ' ').rjust(angleSpaces, ' ') 

    #If no angle then just pad the string until the next argument
    else:
        string =  string + ''.ljust(angleSpaces, ' ')

    #Same as above
    if element.Rotation > 0:
        string = string +  ((str(element.Rotation)).ljust(rotationSpaces-1, ' ')).rjust(rotationSpaces, ' ') 


    elif element.Rotation < 0:
        string = string + ((str(element.Rotation)).ljust(rotationSpaces, ' '))

    else:   
        string =  string + ''.ljust(rotationSpaces, ' ')

    #If there is a comment add it after the padding
    if element.Comment != '':
        string = string + str(element.Comment)

    return(string)


In [3]:
#Define a drift class
class Drift:
    
    def __init__(self, length, angle = 0, rotation = 0, comment = ''):
        
        #Named according to BEATCH convention and so can be placed in GetBEATCHString() easily
        self.Name = ''.ljust(10, ' ')
        self.Length = length
        self.Code = codes['drift']
        self.Angle = angle
        self.Rotation = rotation
        self.Comment = comment
        self.String = GetBEATCHString(self)

In [4]:
#Define an Element class  -  For anything that has a single length (like a detector but unlike magnets with field and yoke lengths)
class Element:
    
    #Position BEATCH wants, middle of element
    def FindBEATCHPosition(self, positionStart, gap):
        self.Position = positionStart + gap + self.Length/2   
    
    #Position of the end of the element
    def FindEndPosition(self, positionStart, gap):
        self.EndPosition = positionStart + gap + self.Length   
    
    #Initialise the class. Element type = standard name, position start = end of last element, gap = drift from last element
    #specialName to overwrite the standard naming and put no position etc
    def __init__(self, elemType, positionStart, gap, angle = 0, rotation = 0, comment = '', specialName = None):
                
        self.Length = lengths[elemType]
        self.FindBEATCHPosition(positionStart, gap)
        self.FindEndPosition(positionStart, gap)
        
        #Change the name if SpecialName exists
        if specialName is None:
            self.Name = elemType.ljust(4, ' ') + '021' + ('%i'%(round(self.Position ))).rjust(3, '0')
        else:
            self.Name = specialName.ljust(10, ' ')
        self.Code = codes[elemType]
        self.Angle = angle
        self.Rotation = rotation
        self.Comment = comment
        self.String = GetBEATCHString(self)

In [5]:
#Define a Magnet class  -  For anything that has a 2 lengths (e.g. field and yoke lengths)
class Magnet:
    
    #Position BEATCH wants, middle of magnet's field with correction for yokes 
    def FindBEATCHPosition(self, positionStart, gap):
        self.Position = positionStart + gap + self.Length/2 + self.LengthYoke
        
    #Position of the end of the magnet
    def FindEndPosition(self, positionStart, gap):
        self.EndPosition = positionStart + gap + self.Length + 2 * self.LengthYoke
        
    #Initialise the class. Element type = standard name, position start = end of last element, gap = drift from last element
    #specialName to overwrite the standard naming and put no position etc
    def __init__(self, elemType, positionStart, gap, angle = 0, rotation = 0, comment = '', specialName = None):
                
        #length of the yoke = (total length  - field length) /2
        self.LengthYoke = (lengths[elemType][0] - lengths[elemType][1])/2
        #length of magnetic field
        self.Length = lengths[elemType][1]
        self.FindBEATCHPosition(positionStart, gap)
        self.FindEndPosition(positionStart, gap)
        
        #Create 2 drifts for before and after the magnet respectively
        LDrift = Drift(self.LengthYoke)
        RDrift = Drift(self.LengthYoke)
        
        #Change the name if SpecialName exists
        if specialName is None:
            self.Name = elemType.ljust(4, ' ') + '021' + ('%i'%(round(self.Position ))).rjust(3, '0')
        else:
            self.Name = specialName.ljust(10, ' ')
            
        self.Code = codes[elemType]
        self.Angle = angle
        self.Rotation = rotation
        self.Comment = comment
        
        #Get the 3 strings needed for the element
        StringLeftYoke = LDrift.String
        StringCore = GetBEATCHString(self)
        StringRightYoke = RDrift.String
        
        #Put the 3 strings together for a full string
        self.String = StringLeftYoke + '\n' + StringCore + '\n' + StringRightYoke

In [14]:
#Distances between my elements

d1= 3.507353
d2= 0.4
d3= 0.45100000000000007
d4= 4.33
d5= 1.83
d6=0.370
d7=0.45100000000000007
d8=1.5873680000000001
d9=0.8
d10=8.204999999999998


#d1= 3.791
#d2= 1.157
#d3= 0.255
#d4= 4.180
#d5= 0.750-0.360
#d6= 0.750+0.360
#d7= 0.295         #B4 and Q3 distance
#d8= 0.399
#d9= 0.399
#d10= 1.991

#d11 = 8.265

theta = 0.0806298

#theta = 0.08345


#Type Magnet, the position of the end of the last magnet, the distance between elements, length of magnet
D0 = Drift(6.467, comment='Q16 to tgt')  ##Drift to align h2vle to old line
T0 = Element('TGT', 480.791, D0.Length, specialName='VLETGT', comment='VLEStart') #Position of Q16 + drift to have line at correct pos
D1 = Drift(d1)
Q1 = Magnet('QPL', round(T0.EndPosition, 3) , D1.Length, comment='Q1 VLE Line' )
D2 = Drift(d2)
Q2 = Magnet('QPL', round(Q1.EndPosition, 3), D2.Length)
D3 = Drift(d3)
B1 = Magnet('MBPL', round(Q2.EndPosition, 3), D3.Length,  angle = theta, rotation=0.014,comment='Start Acrom VLE' )
D4 = Drift(d4)
B2 = Magnet('MBPL', round(B1.EndPosition, 3), D4.Length, angle =-theta, rotation=0.014)
D5 = Drift(d5)
coll = Element('XCSH', round(B2.EndPosition, 3), D5.Length, comment='VLE collimator')
D6 = Drift(d6)
B3 = Magnet('MBPL', round(coll.EndPosition, 3), D6.Length, angle =-theta, rotation=0.014)
D7 = Drift(d4)
B4 = Magnet('MBPL', round(B3.EndPosition, 3), D7.Length, angle =theta, rotation=0.014, comment='End Acrom VLE')
D8 = Drift(d7)
Q3 = Magnet('QPL', round(B4.EndPosition, 3), D8.Length)
D9 = Drift(d8)
Q4 = Magnet('QPL', round(Q3.EndPosition, 3), D9.Length)
D10 = Drift(d9)
Q5 = Magnet('QPL', round(Q4.EndPosition, 3), D10.Length)
D11 = Drift(d10)
#Q6 = Magnet('QPL', round(Q5.EndPosition, 3), D11.Length)
#D12 = Drift(d11)
NA61Tgt = Element('END', round(Q5.EndPosition, 3), D11.Length, comment='NA61TAR')

line = [D0, T0, D1, Q1, D2, Q2, D3, B1, D4, B2, D5, coll, D6, B3, D7, B4, D8, Q3, D9, Q4, D10, Q5, D11, NA61Tgt ]

In [15]:
header = '  BEAM H2VLE Configuration - 23/07/2021, C. A. Mussolini'
mainBeatch = ''
finalLine = '''
GE         
669.23414 4590.727902441.5742 1.454246  -0.000370        
ST'''

counter = 0
for element in line:
    
    mainBeatch = mainBeatch + element.String + '\n'
    if type(element)==Magnet:
        counter +=3
    else: counter +=1
    
    
    
BeatchAddition = header + '\n  ' + str(counter) + '\n' + mainBeatch + finalLine

print(BeatchAddition)

  BEAM H2VLE Configuration - 23/07/2021, C. A. Mussolini
  42
                   1  6.467                                  Q16 to tgt
VLETGT            20  0.200                                  VLEStart
                   1  3.507                                  
                   1  0.230                                  
QPL 021492         2  2.000                                  Q1 VLE Line
                   1  0.230                                  
                   1  0.400                                  
                   1  0.230                                  
QPL 021495         2  2.000                                  
                   1  0.230                                  
                   1  0.451                                  
                   1  0.335                                  
MBPL021498         8  2.000    0.0806298 0.014               Start Acrom VLE
                   1  0.335                                  
                   1  4.33

In [7]:
header = '  BEAM H2VLE Configuration - 23/07/2021, C. A. Mussolini'
mainBeatch = ''
finalLine = '''
GE         
669.23414 4590.727902441.5742 1.454246  -0.000370        
ST'''

counter = 0
for element in line:
    
    mainBeatch = mainBeatch + element.String + '\n'
    if type(element)==Magnet:
        counter +=3
    else: counter +=1
    
    
    
BeatchAddition = header + '\n  ' + str(counter) + '\n' + mainBeatch + finalLine

print(BeatchAddition)

  BEAM H2VLE Configuration - 23/07/2021, C. A. Mussolini
  46
                   1  4.080                                  VLE Drift to align
VLETGT            20  0.200                                  VLEStart
                   1  3.791                                  
                   1  0.230                                  
QPL 021487         2  2.000                                  Q1 VLE Line
                   1  0.230                                  
                   1  1.157                                  
                   1  0.230                                  
QPL 021491         2  2.000                                  
                   1  0.230                                  
                   1  0.205                                  
                   1  0.335                                  
MBPL021493         8  2.000   -0.083381  0.014               Start Acrom VLE
                   1  0.335                                  
                  

In [14]:
newFileText = BeforeSplit + BeatchAddition + AfterSplit

print(BeforeSplit + BeatchAddition + AfterSplit)

  BEAM H2VLE Configuration - 23/07/2021, C. A. Mussolini
  46
                   1  4.080                                  VLE Drift to align
VLETGT            20  0.200                                  VLEStart
                   1  3.791                                  
                   1  0.230                                  
QPL 021487         2  2.000                                  Q1 VLE Line
                   1  0.230                                  
                   1  1.157                                  
                   1  0.230                                  
QPL 021491         2  2.000                                  
                   1  0.230                                  
                   1  0.105                                  
                   1  0.335                                  
MBPL021493         8  2.000   -0.083381  0.014               Start Acrom VLE
                   1  0.335                                  
                  

In [54]:
Q2.EndPosition

491.94899999999996

In [11]:
BeforeSplit = ''
AfterSplit = ''